In [ ]:
SET ANSI_NULLS ON

In [ ]:
SET QUOTED_IDENTIFIER ON

In [ ]:
DROP VIEW IF EXISTS [dbo].[ParseFields]

In [ ]:
CREATE VIEW [dbo].[ParseFields]
AS
Select 
    A.Id
    -- ,B.RetSeq
    -- ,B.RetVal
    -- , B.*
    -- , C.Pos6 AS 'Level'
    , C.Pos3 AS 'Category'
    , C.Pos5 AS 'TimeStamp'
    , C.Pos6 AS 'EventID'
    , C.Pos7 AS 'Windows Log'
    , C.Pos10 AS 'Short Message'
    , C.Pos11 AS 'Host Name'
    , C.Pos13 AS 'Message Subject'
From dbo.Logs1 A
 Cross Apply [dbo].[udf-Str-Parse](A.Message,char(13)) B
 Cross Apply [dbo].[udf-Str-Parse-Row](B.RetVal,char(9)) C
Where B.RetVal is not null and B.RetSeq = 1

In [ ]:
IF OBJECT_ID('[dbo].[ParsedFieldValues]', 'U') IS NOT NULL
DROP TABLE [dbo].[ParsedFieldValues]

In [ ]:
CREATE TABLE [dbo].[ParsedFieldValues]
(
      [Id] INT NOT NULL PRIMARY KEY
    , [Category] NVARCHAR(255) NOT NULL
    , [TimeStamp] NVARCHAR(255) NOT NULL
    , [EventID] INT NOT NULL
    , [Windows Log] NVARCHAR(255) NOT NULL
    , [Short Message] NVARCHAR(255) NULL
    , [Host Name] NVARCHAR(255) NOT NULL
    , [Message Subject] NVARCHAR(255) NOT NULL
);

In [ ]:
CREATE NONCLUSTERED INDEX
    ID_Category
    ON [LabLog].[dbo].[ParsedFieldValues] ([Category])

In [ ]:
CREATE NONCLUSTERED INDEX
    ID_HostName
    ON [LabLog].[dbo].[ParsedFieldValues] ([Host Name])

In [ ]:
CREATE NONCLUSTERED INDEX
    ID_WindowsLog
    ON [LabLog].[dbo].[ParsedFieldValues] ([Windows Log])

In [ ]:
CREATE NONCLUSTERED INDEX
    ID_EventID
    ON [LabLog].[dbo].[ParsedFieldValues] ([EventID])

In [ ]:
CREATE NONCLUSTERED INDEX
    ID_ShortMessage
    ON [LabLog].[dbo].[ParsedFieldValues] ([Short Message])

In [ ]:
TRUNCATE TABLE dbo.ParsedFieldValues

In [ ]:
INSERT INTO dbo.ParsedFieldValues
SELECT 
  [Id] 
, [Category] 
, [TimeStamp]
, CONVERT(INT, [EventID]) as 'EventID'
, [Windows Log]
, [Short Message]
, [Host Name]
, [Message Subject]
FROM dbo.ParseFields

In [ ]:
MERGE dbo.ParsedFieldValues t
USING dbo.ParseFields s
ON (t.[Id] = s.[Id])
WHEN MATCHED
    THEN UPDATE 
        SET 
      t.[Category] = s.[Category]
    , t.[TimeStamp] = s.[TimeStamp]
    , t.[EventID] = s.[EventID]
    , t.[Windows Log] = s.[Windows Log]
    , t.[Short Message] = s.[Short Message]
    , t.[Host Name] = s.[Host Name]
    , t.[Message Subject] = s.[Message Subject]
WHEN NOT MATCHED BY TARGET
    THEN INSERT (
      [Id] 
    , [Category] 
    , [TimeStamp] 
    , [EventID] 
    , [Windows Log]
    , [Short Message]
    , [Host Name]
    , [Message Subject]
    ) 
    VALUES (
      s.[Id] 
    , s.[Category] 
    , s.[TimeStamp] 
    , s.[EventID] 
    , s.[Windows Log]
    , s.[Short Message]
    , s.[Host Name]
    , s.[Message Subject]
    )
WHEN NOT MATCHED BY SOURCE 
THEN DELETE;